In [1]:
import numpy as np
import pandas as pd
import zipfile
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import category_encoders as ce

In [2]:
pd.set_option('display.max_columns', 500)
max_abs_scaler = MaxAbsScaler()

le = preprocessing.LabelEncoder()

zf = zipfile.ZipFile('../data/raw_data/student_exams.zip')
print(zipfile.ZipFile.namelist(zf)[0])
data = pd.read_csv(
    zf.open(zipfile.ZipFile.namelist(zf)[0]))
data

StudentsPerformance.csv


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [3]:
data_target = data['math score']
data_features = data.drop(columns=['math score', 'reading score', 'writing score'])

In [6]:
def to_small_score(x):
    return int(x/20)

In [7]:
data_target = data_target.apply(to_small_score)

In [10]:
for column_name in data_features.columns:
    data_features[column_name] = data_features[column_name].astype('category')
    data_features[column_name] = data_features[column_name].cat.codes


encoder = ce.OneHotEncoder(cols=data_features.columns, return_df=True)

data_features_encoded = encoder.fit_transform(data_features)

target_encoded = le.fit_transform(data_target)

data_features_encoded['target']=target_encoded
data_features_encoded

,gender_1,gender_2,race/ethnicity_1,race/ethnicity_2,race/ethnicity_3,race/ethnicity_4,race/ethnicity_5,parental level of education_1,parental level of education_2,parental level of education_3,parental level of education_4,parental level of education_5,parental level of education_6,lunch_1,lunch_2,test preparation course_1,test preparation course_2,target
0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,3
1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,3
2,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,4
3,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,2
4,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,4
996,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,3
997,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,2
998,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,3


In [11]:
compression=dict(
    method='zip', archive_name='d22 - student_exams.csv')

data_features_encoded.to_csv(
    '../data/preprocessed_data/d22 - student_exams.zip', index=False, compression=compression)